In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pybdshadow

In [6]:
buildings = gpd.read_file(r'../example/data/Suzhoubuildings/苏州(1).shp')
buildings = pybdshadow.bd_preprocess(buildings)

In [2]:
buildings = gpd.read_file(r'../example/data/bd_demo.json')
buildings = pybdshadow.bd_preprocess(buildings)

In [3]:
buildings = buildings.iloc[:10000]

In [3]:
from pybdshadow import *
date = pd.to_datetime('2015-01-01 02:45:33.959797119')
height = 'height'
ground = 0
epsg = 3857

shadows = pybdshadow.bdshadow_pointlight(buildings)
#shadows = pybdshadow.bdshadow_sunlight(buildings, date,merge = True)

In [4]:
pybdshadow.show_bdshadow(buildings = buildings,shadows = shadows)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '4eo0v3', 'type': '…